In [2]:
from src.data.dataset_functions import get_dataframei_onlymods

modframe = get_dataframei_onlymods(78)

modframe.show(5)

providing ../data/raw/2022_place_canvas_history-000000000078.csv ...
not found. need to download ../data/raw/2022_place_canvas_history-000000000078.csv.gzip ...
downloading from https://placedata.reddit.com/data/canvas-history/2022_place_canvas_history-000000000078.csv.gzip to ../data/raw/2022_place_canvas_history-000000000078.csv.gzip
unpacking ../data/raw/2022_place_canvas_history-000000000078.csv.gzip into ../data/raw/2022_place_canvas_history-000000000078.csv
deleting ../data/raw/2022_place_canvas_history-000000000078.csv.gzip


+-------+---+---+---+---+---+-----------+
|user_id| x1| y1| x2| y2|  t|pixel_color|
+-------+---+---+---+---+---+-----------+
+-------+---+---+---+---+---+-----------+



Na dann gucken wir mal was bei (271,1835)-(296,1859) zum Zeitpunkt 562 übermalt wurde

In [29]:
from src.data.dataset_functions import get_dataframei

dataFrame = get_dataframei(3)

providing ../data/raw/2022_place_canvas_history-000000000003.csv ...
../data/raw/2022_place_canvas_history-000000000003.csv is already in data/raw


In [30]:
min_x = 271
max_x = 296
min_y = 1835
max_y = 1859
tz = 562

In [31]:
from src.features.feature_functions import get_latestpixels_from_box

censoredData = get_latestpixels_from_box(dataFrame,min_x,min_y,max_x,max_y,tz)

In [32]:
censoredData.show(5)
censoredData.count()

+---+----+--------------------+---+-----------+
|  x|   y|             user_id|  t|pixel_color|
+---+----+--------------------+---+-----------+
|272|1859|z90rX9xnUC9Tfb/T7...|359|    #000000|
|273|1853|GuF+xzrhhROmfacr6...|147|    #FF4500|
|274|1852|HHV4dR8dNHXWZsQjL...| 27|    #FF4500|
|274|1857|I35ca7LfMboIZJhOz...|328|    #FF99AA|
|275|1843|L1axPsUn4/6uJMpN3...|556|    #FFF8B8|
+---+----+--------------------+---+-----------+
only showing top 5 rows



151

Koordinaten normalisieren...

In [33]:
censoredData = censoredData.withColumn('x' , censoredData['x'] - min_x).withColumn('y', censoredData['y'] - min_y)
censoredData.show(5)

+---+---+--------------------+---+-----------+
|  x|  y|             user_id|  t|pixel_color|
+---+---+--------------------+---+-----------+
|  1| 24|z90rX9xnUC9Tfb/T7...|359|    #000000|
|  2| 18|GuF+xzrhhROmfacr6...|147|    #FF4500|
|  3| 17|HHV4dR8dNHXWZsQjL...| 27|    #FF4500|
|  3| 22|I35ca7LfMboIZJhOz...|328|    #FF99AA|
|  4|  8|L1axPsUn4/6uJMpN3...|556|    #FFF8B8|
+---+---+--------------------+---+-----------+
only showing top 5 rows



In [34]:
len_x = max_x-min_x+1
print(len_x)
len_y = max_y-min_y+1
print(len_y)

26
25


In [35]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [36]:
def hex_to_rbg(hexcolor):
    hexcolor = hexcolor.lstrip('#')
    lv = len(hexcolor)
    return [int(hexcolor[i:i + lv // 3], 16) for i in range(0, lv, lv // 3)]

def hex_to_float(hexcolor):
    hexcolor = hexcolor.lstrip('#')
    lv = len(hexcolor)
    return [int(hexcolor[i:i + lv // 3], 16) / 255.0 for i in range(0, lv, lv // 3)]


In [37]:
data = np.zeros([len_x,len_y,3])

collectedData = censoredData.collect()

for row in collectedData:
    xs , ys , user_id , ts , hexcolor = list(row)
    data[xs][ys] = hex_to_float(hexcolor)

In [38]:
mpimg.imsave("test.jpg",data)